In [1]:
import cv2
import glob
import numpy as np
from skimage.feature import peak_local_max
from skimage.morphology import watershed
from scipy import ndimage
import csv

## Getting all the images

In [2]:
images = glob.glob("./retail_images/*.*")
escape=False

## Steps taken to get the correct contours 

1. Sort the contours in the descending order of areas
2. Iterate through sorted contours and compare the area of a contour with previous contour
3. If the contour area is greater than 80% of previous contours continue
4. Else, stop the process.
5. With the filtered contous create a convex hull and return it.


This function also identifies if this image has to be processed through OpenCV or FasterRCNN
If the area of a contour is greater than 50% of the whole image then pass a flag of large contour
which indicate that this image has to be processed through FasterRCNN Algorithm


In [3]:

def get_correct_contours(contours,area):
    contour_area_list = []
    large_contour = False
    cnts = sorted(contours, key = cv2.contourArea, reverse = True)
    for i in cnts:
        if cv2.contourArea(i) < 0.5*area:
            contour_area_list.append(cv2.contourArea(i))
        else:
            if large_contour==False:
                large_contour = True
    prev_area=1.0
    for j,k in enumerate(contour_area_list):
        if j !=0:
            if prev_area != 0:
                if (prev_area-k)/prev_area > 0.8:
                    break
        else:
            prev_area = k
    convex_hull = []
    if prev_area != 1.0:
        if j !=1:
            length = j+1
        else:
            length = j
        for element in cnts[0:length]:
            convex_hull.append(cv2.convexHull(element)) 
    return(convex_hull,large_contour)

In [ ]:
csvfile = open("OpenCV.csv", 'w')
writer = csv.writer(csvfile)
for img in images:
    kernel = np.ones((2,2),np.uint8)
    if escape==True:
        cv2.destroyAllWindows()
        break
    else:
        #Read the Image in color format
        color=cv2.imread(img,1)
        #Covert the color image to gray scale
        gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)
        
        #Threshold the image using Otsu
        ret3,th3 = cv2.threshold(gray,0,255,cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

        # Create the minimum and maximum threshold values for Canny Edge detection
        hist_eq =cv2.equalizeHist(gray)
        min_thresh=0.66*np.median(hist_eq)
        max_thresh=1.33*np.median(hist_eq)
        

        edge_1 = cv2.Canny(gray,min_thresh,max_thresh)
        
        # Dilate the edges to make it more visible
        edge_2 = cv2.dilate(edge_1,None,iterations = 1)
        
        #Flood fill the image to identify the background and extract foreground
        im_floodfill = edge_2.copy()
        h, w = im_floodfill.shape[:2]
        area = h*w
        mask = np.zeros((h+2, w+2), np.uint8)
 
        #Floodfill from point (0, 0)
        cv2.floodFill(im_floodfill, mask, (0,0), 255)
        im_floodfill_inv = cv2.bitwise_not(im_floodfill)
        im_out = edge_2 | im_floodfill_inv
        
        
        # Find all external contours
        im2, contours, hierarchy = cv2.findContours(im_out,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        
        # Get the correct contours.       
        cnts,large_contour = get_correct_contours(contours,area)
        
        
        # If the contour is not large 
        if not large_contour and len(cnts) >2:
                draw_contour_all = cv2.drawContours(color, cnts, -1, (0,255,0), 3)
                outer = cv2.drawContours(color,cnts,-1,(0,255,0), 3)
                writer.writerow([img.rsplit('/', 1)[1].split('.')[0],len(cnts)])
                print(img.rsplit('/', 1)[1].split('.')[0],len(cnts))
        else:
            writer.writerow([img.rsplit('/', 1)[1].split('.')[0],'FasterRCNN'])
csvfile.close()